Plan d’organisation pour la Partie B, inspiré de la Partie A, pour entraîner un classifieur de priorité avec BARThez.  


---

## 1. Membres de l’équipe
   
- Demba Seck secd1700  
- Cheikh Tidiane Gueye
- Fatou Bintou Boye boyf7012 


---

## 2. Télécharger et préparer le jeu de données


In [1]:
import urllib.request

url = 'https://drive.google.com/uc?export=download&id=13ZfF8DjSvqPJkrY93GJTK2l14IwLVLZ0'
urllib.request.urlretrieve(url, 'IT_Support_Tickets_FR_200.csv')
print('Téléchargement terminé !')

Téléchargement terminé !




---

## 3. Charger le DataFrame et afficher les colonnes


In [2]:
import pandas as pd
df = pd.read_csv("IT_Support_Tickets_FR_200.csv")
print(df.columns)
df.head()

Index(['ID Ticket', 'Date', 'Client', 'Description', 'Type de problème',
       'Priorité'],
      dtype='object')


,ID Ticket,Date,Client,Description,Type de problème,Priorité
0,1,2023-10-01,Service Finances,L'ordinateur ne s'allume pas après une mise à ...,Matériel,Élevée
1,2,2023-10-01,Marketing,"Impossible d'accéder à Outlook - erreur ""Compt...",Compte/Mot de passe,Moyenne
2,3,2023-10-02,RH,Le logiciel SAP plante fréquemment lors de la ...,Logiciel,Élevée
3,4,2023-10-02,IT Support,Connexion Wi-Fi lente dans le bâtiment B,Réseau,Moyenne
4,5,2023-10-03,Direction,Impossible d'imprimer depuis le nouveau copieur,Matériel,Élevée




---

## 4. Préparer les données pour la classification de priorité


In [3]:
# On retire les colonnes inutiles
colonnes_a_retirer = ['ID Ticket', 'Date', 'Client', 'Type de problème']
df_priorite = df.drop(columns=colonnes_a_retirer)
df_priorite.head()

,Description,Priorité
0,L'ordinateur ne s'allume pas après une mise à ...,Élevée
1,"Impossible d'accéder à Outlook - erreur ""Compt...",Moyenne
2,Le logiciel SAP plante fréquemment lors de la ...,Élevée
3,Connexion Wi-Fi lente dans le bâtiment B,Moyenne
4,Impossible d'imprimer depuis le nouveau copieur,Élevée




---

## 5. Encoder la priorité


In [4]:
from sklearn.preprocessing import LabelEncoder

encodeur_priorite = LabelEncoder()
df_priorite['Classe'] = encodeur_priorite.fit_transform(df_priorite['Priorité'])
df_priorite.head()

,Description,Priorité,Classe
0,L'ordinateur ne s'allume pas après une mise à ...,Élevée,2
1,"Impossible d'accéder à Outlook - erreur ""Compt...",Moyenne,1
2,Le logiciel SAP plante fréquemment lors de la ...,Élevée,2
3,Connexion Wi-Fi lente dans le bâtiment B,Moyenne,1
4,Impossible d'imprimer depuis le nouveau copieur,Élevée,2




---

## 6. Split train/val/test


In [5]:
from sklearn.model_selection import train_test_split

X = df_priorite['Description']
y = df_priorite['Classe']

X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)

print(len(X_train), len(X_val), len(X_test))

120 40 40




---

## 7. Tokenizer BARThez


In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("moussaKam/barthez")
X_train_list = X_train.tolist()
X_val_list = X_val.tolist()
X_test_list = X_test.tolist()

train_tokens = tokenizer(X_train_list, padding=True, truncation=True, return_tensors="pt")
val_tokens = tokenizer(X_val_list, padding=True, truncation=True, return_tensors="pt")
test_tokens = tokenizer(X_test_list, padding=True, truncation=True, return_tensors="pt")

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm




---

## 8. Dataset PyTorch


In [7]:
from torch.utils.data import Dataset
import torch

class TicketDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item

train_dataset = TicketDataset(train_tokens, y_train.tolist())
val_dataset = TicketDataset(val_tokens, y_val.tolist())
test_dataset = TicketDataset(test_tokens, y_test.tolist())



---

## 9. Modèle BARThez pour classification


In [8]:
from transformers import AutoModelForSequenceClassification

n_classes = len(df_priorite['Classe'].unique())
model = AutoModelForSequenceClassification.from_pretrained("moussaKam/barthez", num_labels=n_classes)

Some weights of MBartForSequenceClassification were not initialized from the model checkpoint at moussaKam/barthez and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.




---

## 10. Entraînement du modèle


In [9]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=5,
    weight_decay=0.01,
    logging_dir="./logs",
)

import numpy as np
from sklearn.metrics import accuracy_score, f1_score

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    if isinstance(predictions, tuple):
        predictions = predictions[0]
    preds = np.argmax(predictions, axis=1)
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds, average="weighted")
    }

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)



---

## 11. Lancer l’entraînement


In [ ]:
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:



---

## 12. Évaluer sur le jeu de test


In [ ]:
test_metrics = trainer.evaluate(test_dataset)
print(test_metrics)



---

## 13. Afficher les prédictions (optionnel)


In [ ]:
predictions = trainer.predict(test_dataset)
logits = predictions.predictions
if isinstance(logits, tuple):
    logits = logits[0]
pred_labels = np.argmax(logits, axis=1)
true_labels = predictions.label_ids

for i in range(len(test_dataset)):
    print("Description:", X_test.iloc[i])
    print("True label:", encodeur_priorite.inverse_transform([true_labels[i]])[0])
    print("Predicted:", encodeur_priorite.inverse_transform([pred_labels[i]])[0])
    print("-" * 50)



---

## 14. Résultats (cellule texte)
Indiquez ici le score F1 obtenu sur le jeu de test.

---

Adapte chaque cellule selon tes besoins.  
N’oublie pas d’expliquer chaque étape dans des cellules texte !